In [4]:
from typing import Optional
import psycopg


def execute_query(query: str, params: Optional[dict]) -> list[tuple]:
    with psycopg.connect(
        **{
            "dbname": "comp0022",
            "user": "admin",
            "password": "top_secret_password_db",
            "host": "localhost",
            "port": "5432",
        }
    ) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, params)
            results = cursor.fetchall()

    conn.close()
    return results

In [28]:
query = """
    WITH
        relevant_users AS (
            SELECT
                r.user_id
            FROM
                ratings r
            WHERE
                r.movie_id = %(movie_id)s
            GROUP BY
                r.user_id
            ORDER BY
                RANDOM()
            LIMIT
                5
        ),
        kindness_bias AS (
            SELECT
                (
                    SELECT
                        AVG(avg_u_rating)
                    FROM (
                        SELECT
                            AVG(r.rating) AS avg_u_rating
                        FROM
                            ratings r
                        GROUP BY
                            r.user_id
                    )
                ) / AVG(avg_ru_rating)
            FROM (
                SELECT
                    AVG(r.rating) AS avg_ru_rating
                FROM
                    ratings r
                INNER JOIN
                    relevant_users ru ON r.user_id = ru.user_id
                GROUP BY
                    ru.user_id
            )
        ),
        genre_bias AS (
            SELECT
                (
                    SELECT
                        AVG(r.rating)
                    FROM
                        ratings r
                    INNER JOIN
                        movies_genres mg ON r.movie_id = mg.movie_id
                    WHERE
                        mg.genre_id IN (
                            SELECT
                                mg.genre_id
                            FROM
                                movies_genres mg
                            WHERE
                                mg.movie_id = %(movie_id)s
                        )
                ) / AVG(r.rating)
            FROM
                ratings r
            INNER JOIN
                relevant_users ru ON r.user_id = ru.user_id
            INNER JOIN
                movies_genres mg ON r.movie_id = mg.movie_id
            WHERE
                mg.genre_id IN (
                    SELECT
                        mg.genre_id
                    FROM
                        movies_genres mg
                    WHERE
                        mg.movie_id = %(movie_id)s
                )
        ),
        tag_bias AS (
            SELECT
                (
                    SELECT
                        AVG(r.rating)
                    FROM
                        ratings r
                    INNER JOIN
                        movies_users_tags mut ON r.movie_id = mut.movie_id AND r.user_id = mut.user_id
                    WHERE
                        mut.tag_id IN (
                            SELECT
                                DISTINCT mut.tag_id
                            FROM
                                movies_users_tags mut
                            WHERE
                                mut.movie_id = %(movie_id)s
                        )
                ) / AVG(r.rating)
            FROM
                ratings r
            INNER JOIN
                relevant_users ru ON r.user_id = ru.user_id
            INNER JOIN
                movies_users_tags mut ON r.movie_id = mut.movie_id AND r.user_id = mut.user_id
            WHERE
                mut.tag_id IN (
                    SELECT
                        DISTINCT mut.tag_id
                    FROM
                        movies_users_tags mut
                    WHERE
                        mut.movie_id = %(movie_id)s
                )
        )
    SELECT
        AVG(r.rating),
        (
            SELECT
                AVG(r.rating)
            FROM
                ratings r
            INNER JOIN
                relevant_users ru ON r.user_id = ru.user_id
            WHERE
                r.movie_id = %(movie_id)s
        ),
        (SELECT * FROM kindness_bias),
        (SELECT * FROM genre_bias),
        (SELECT * FROM tag_bias)
    FROM
        ratings r
    WHERE r.movie_id = %(movie_id)s
    ;
"""

execute_query(query, {"movie_id": 1})

[(3.9209302325581397,
  3.4,
  0.9637732298914251,
  0.9803013650244932,
  1.0178571428571428)]

In [53]:
import time


queries = {
    "join": """
SELECT
                m.image_url,
                m.title,
                m.year,
                AVG(r.rating),
                ARRAY_AGG(DISTINCT g.genre) AS genres,
                ARRAY_AGG(DISTINCT t.tag) AS tags,
                ARRAY_AGG(r.rating) FILTER (WHERE r.rating IS NOT NULL) AS ratings,
                ARRAY_AGG(DISTINCT a.name) as actors,
                ARRAY_AGG(DISTINCT d.name) as directors
            FROM movies m
            LEFT JOIN ratings r ON m.movie_id = r.movie_id
            LEFT JOIN movies_genres mg ON m.movie_id = mg.movie_id
            LEFT JOIN genres g ON mg.genre_id = g.genre_id
            LEFT JOIN tags t ON m.movie_id = t.movie_id
            left join movies_actors ma on m.movie_id = ma.movie_id
            left join actors a on ma.actor_id = a.actor_id
            left join movies_directors md on m.movie_id = md.movie_id
            left join directors d on md.director_id = d.director_id
            WHERE 1=1
            GROUP BY m.title, m.image_url, m.year
            ORDER BY AVG(r.rating) DESC
            LIMIT 15
""",
    "inner_join_minimal_return": """
SELECT
                m.image_url,
                m.title,
                m.year
            FROM movies m
            inner JOIN ratings r ON m.movie_id = r.movie_id
            inner JOIN movies_genres mg ON m.movie_id = mg.movie_id
            inner JOIN genres g ON mg.genre_id = g.genre_id
            inner JOIN tags t ON m.movie_id = t.movie_id
            inner join movies_actors ma on m.movie_id = ma.movie_id
            inner join actors a on ma.actor_id = a.actor_id
            inner join movies_directors md on m.movie_id = md.movie_id
            inner join directors d on md.director_id = d.director_id
            WHERE 1=1
            GROUP BY m.title, m.image_url, m.year
            ORDER BY AVG(r.rating) DESC
            LIMIT 5
""",
    "minimal": """
SELECT
                m.image_url,
                m.title,
                m.year
            FROM movies m
            inner JOIN ratings r ON m.movie_id = r.movie_id
            WHERE 1=1
            GROUP BY m.title, m.image_url, m.year
            ORDER BY AVG(r.rating) DESC
            LIMIT 5
""",
    "minimal_no_order": """
SELECT
                m.image_url,
                m.title,
                m.year
            FROM movies m
            inner JOIN ratings r ON m.movie_id = r.movie_id
            WHERE 1=1
            GROUP BY m.title, m.image_url, m.year
            LIMIT 5
""",
}

params = {}

times = {}
for n, q in queries.items():
    best_time = 100
    for _ in range(3):
        start_time = time.time()
        execute_query(q, params)
        end_time = time.time()
        best_time = min(best_time, end_time - start_time)
    times[n] = best_time
times

{'inner_join_minimal_return': 1.2075157165527344,
 'minimal': 0.05252504348754883,
 'minimal_no_order': 0.049154043197631836}

In [42]:
import concurrent.futures

best_time = 100
for _ in range(30):
    start_time = time.time()
    pt_1 = """select dm.movie_id from movies_directors dm
    inner join directors d on d.director_id = dm.director_id
    where lower(d.name) like lower(%(directorname)s)"""

    pt_2 = """select am.movie_id from movies_actors am
    inner join actors a on a.actor_id = am.actor_id
    where lower(a.name) like lower(%(actorname)s)"""

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_query = {
            executor.submit(execute_query, pt_1, {"directorname": "A%"}): "director",
            executor.submit(execute_query, pt_2, {"actorname": "A%"}): "actor",
        }

        result_1 = []
        result_2 = []

        for future in concurrent.futures.as_completed(future_to_query):
            query_type = future_to_query[future]
            if query_type == "director":
                result_1 = future.result()
            elif query_type == "actor":
                result_2 = future.result()

    result_intersection = set(result_1).intersection(set(result_2))
    end_time = time.time()

    best_time = min(best_time, end_time - start_time)
best_time

0.022302865982055664

In [43]:
best_time = 100
for _ in range(30):
    start_time = time.time()
    query = """
    select * from movies m
    left join movies_directors dm on dm.movie_id = m.movie_id
    left join movies_actors am on am.movie_id = m.movie_id
    left join directors d on d.director_id = dm.director_id
    left join actors a on a.actor_id = am.actor_id
    where a.name like %(actorname)s
    and d.name like %(directorname)s"""

    params = {"actorname": "A%", "directorname": "A%"}
    execute_query(query, params)
    end_time = time.time()
    best_time = min(best_time, end_time - start_time)
best_time

0.00890207290649414